In [1]:
# # !pip uninstall jax -y
# # !pip install jax
# incase you get error for tf wrt jax
# we can use this to deploy

In [2]:
# !pip install stable-baselines3[extra]
# from stable_baselines3 import PPO
# import gym

In [3]:
!pip install fastapi

**TODO**:
* Lets save the model with hdf5 (h5)
* Lets create a FAST API that will allow our model to interact with the outside world by receiving input data, processing it through the model, and returning predictions. FLASK is simple but we will be using FASTAPI as it can be easly deployed with AWS.
* Integrate the ML model and API 



**References**:

* Dataset : https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
* Reinforcement learning paper : https://arxiv.org/pdf/2112.04236.pdf 
* Ref for auto encoders : https://www.kaggle.com/code/robinteuwens/anomaly-detection-with-auto-encoders 
* FASTAPI : https://www.kaggle.com/code/abbascanguven/python-deploying-an-ml-model-with-fastapi

In [4]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.metrics import mean_squared_error, accuracy_score
from keras import regularizers
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,recall_score

In [5]:
  for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/credit-card-fraud-detection-dataset-2023/creditcard_2023.csv


In [6]:
data=pd.read_csv('/kaggle/input/credit-card-fraud-detection-dataset-2023/creditcard_2023.csv')
print("shape of the data: ",data.shape)

shape of the data:  (568630, 31)


In [7]:
headlist =[]
print(data.head())
headlist = data.columns.tolist()
print('data.shape', data.shape)
# headlist

   id        V1        V2        V3        V4        V5        V6        V7  \
0   0 -0.260648 -0.469648  2.496266 -0.083724  0.129681  0.732898  0.519014   
1   1  0.985100 -0.356045  0.558056 -0.429654  0.277140  0.428605  0.406466   
2   2 -0.260272 -0.949385  1.728538 -0.457986  0.074062  1.419481  0.743511   
3   3 -0.152152 -0.508959  1.746840 -1.090178  0.249486  1.143312  0.518269   
4   4 -0.206820 -0.165280  1.527053 -0.448293  0.106125  0.530549  0.658849   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0 -0.130006  0.727159  ... -0.110552  0.217606 -0.134794  0.165959  0.126280   
1 -0.133118  0.347452  ... -0.194936 -0.605761  0.079469 -0.577395  0.190090   
2 -0.095576 -0.261297  ... -0.005020  0.702906  0.945045 -1.154666 -0.605564   
3 -0.065130 -0.205698  ... -0.146927 -0.038212 -0.214048 -1.893131  1.003963   
4 -0.212660  1.049921  ... -0.106984  0.729727 -0.161666  0.312561 -0.414116   

        V26       V27       V28    Amount  C

In [8]:
trues = (data['Class'] == 1).sum()
print(trues)
falses = (data['Class'] == 0).sum()
print(falses)
unique_count = (data['Class'] == 0).unique()
print(unique_count)

# Plotting
# data.plot(kind='bar')  # You can change 'line' to 'bar', 'hist', 'box', etc.
# plt.show()

284315
284315
[ True False]


> Test: shuffle the data and run the model. check the diff. might reduce bias from the pattern.

In [9]:
x = data.drop(['Class', 'id'], axis=1)
y = data['Class']
# print(x,y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

In [10]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(454904, 29) (113726, 29) (454904,) (113726,)


In [11]:
scaler = StandardScaler()
x_train['Amount'] = scaler.fit_transform(x_train[['Amount']])
x_test['Amount'] = scaler.transform(x_test[['Amount']])

In [12]:
# input_dim = x_train.shape[1]

# # Define the layers
# input_layer = Input(shape=(input_dim,))
# encoder = Dense(14, activation="tanh")(input_layer)
# encoder = Dense(7, activation="relu")(encoder)
# decoder = Dense(7, activation='tanh')(encoder)
# decoder = Dense(input_dim, activation='relu')(decoder)

# autoencoder = Model(inputs=input_layer, outputs=decoder)

# # Training accuracy: 0.7691007333415402 
# # Test accuracy: 0.7688655188787085

In [13]:
input_dim = x_train.shape[1]

input_layer = Input(shape=(input_dim,))

encoder = Dense(14, activation="tanh")(input_layer)
encoder = Dense(10, activation="relu")(encoder)
encoder = Dense(7, activation="relu")(encoder)

decoder = Dense(7, activation='tanh')(encoder)
decoder = Dense(10, activation='relu')(decoder)
decoder = Dense(input_dim, activation='relu')(decoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)

# Training accuracy: 0.784978808715685
# Test accuracy: 0.7841126919086224

In [14]:
# input_dim = x_train.shape[1]

# input_layer = Input(shape=(input_dim,))

# print(input_dim)
# # print(input_layer)

In [15]:
# input_dim = x_train.shape[1]

# input_layer = Input(shape=(input_dim,))

# encoded = Dense(100, activation= 'tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
# encoded = Dense(50, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(encoded)
# encoded = Dense(25, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(encoded)
# encoded = Dense(12, activation = 'tanh', activity_regularizer=regularizers.l1(10e-5))(encoded)
# encoded = Dense(6, activation='relu')(encoded)

# # Decoder network
# decoded = Dense(12, activation='tanh')(encoded)
# decoded = Dense(25, activation='tanh')(decoded)
# decoded = Dense(50, activation='tanh')(decoded)
# decoded = Dense(100, activation='tanh')(decoded)

# output_layer = Dense(x.shape[1], activation='relu')(decoded)

# # Building a model
# autoencoder = Model(input_layer, output_layer)

# # Training accuracy: 0.8050182016425443
# # Test accuracy: 0.8044510490125389
# # do not remember number of epochs

In [16]:
x_train_nonfraud = x_train[y_train == 0]

autoencoder.compile(optimizer='adam', loss='mean_squared_error') # can be binary cross
autoencoder.fit(x_train_nonfraud, x_train_nonfraud, epochs=1, batch_size=32, shuffle=True, validation_split=0.15)

6042/6042 [==============================] - 15s 2ms/step - loss: 0.5043 - val_loss: 0.4299


In [17]:
autoencoder.save('autoencoder_model.h5')
autoencoder.save_weights('autoencoder_weights.h5')

# autoencoder.load_weights('autoencoder_weights.h5')

In [18]:
# 1. Calculate the Mean Squared Error (MSE)
train_predictions = autoencoder.predict(x_train)
train_mse = np.mean(np.power(x_train - train_predictions, 2), axis=1)

test_predictions = autoencoder.predict(x_test)
test_mse = np.mean(np.power(x_test - test_predictions, 2), axis=1)

# 2. Determine a Threshold
threshold = np.percentile(train_mse[y_train == 0], 95)

# 3. Evaluate Accuracy
train_pred_labels = [1 if mse > threshold else 0 for mse in train_mse]
test_pred_labels = [1 if mse > threshold else 0 for mse in test_mse]

train_accuracy = accuracy_score(y_train, train_pred_labels)
test_accuracy = accuracy_score(y_test, test_pred_labels)

print("Training accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)

# confusion matrix and recall 

# confusion matrix for training data
train_cm = confusion_matrix(y_train, train_pred_labels)

# confusion matrix for testing data
test_cm = confusion_matrix(y_test, test_pred_labels)

print("Train confusionn matrix : ", train_cm)
print("Test confusion matrix : ", test_cm)
#print(f"Recall: {recall:.2f}")

3554/3554 [==============================] - 5s 1ms/step
Training accuracy: 0.7896523222482106
Test accuracy: 0.7893621511351846
Train confusionn matrix :  [[216079  11373]
 [ 84315 143137]]
Test confusion matrix :  [[53944  2919]
 [21036 35827]]


In [19]:
x_test_predictions = autoencoder.predict(x_test)
mse = np.mean(np.power(x_test - x_test_predictions, 2), axis=1)
print(max(mse))

3554/3554 [==============================] - 4s 1ms/step
179.48441103111122


In [20]:
import logging
import io
from datetime import datetime

def setup_logging(timestamp):
    """Sets up the logging environment."""
    log_filename = f'logs/model_run_{timestamp}.log'
    
    # Ensure the logs directory exists
    if not os.path.exists('logs'):
        os.makedirs('logs')
    
    # Configure logging
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    
    # Check if logger has handlers already (important in Jupyter environments)
    if not logger.handlers:
        file_handler = logging.FileHandler(log_filename)
        file_handler.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)
    
    return logger

def log_model_details(timestamp, model_summary, epochs, training_accuracy, testing_accuracy):
    """
    Logs the model details to the specified log file.
    
    :param timestamp: Timestamp for the log file
    :param model_summary: Summary of the model architecture
    :param epochs: Number of epochs used for training
    :param training_accuracy: The training accuracy
    :param testing_accuracy: The testing accuracy
    """
    logger = setup_logging(timestamp)
    
    # Log the model summary
    logger.info("Model Summary:\n" + model_summary)
    
    # Log the training details
    logger.info(f"Training for {epochs} epochs.")
    logger.info(f"Training Accuracy: {training_accuracy}")
    logger.info(f"Testing Accuracy: {testing_accuracy}")
    logger.info("***************************************")

In [21]:
stream = io.StringIO()
autoencoder.summary(print_fn=lambda x: stream.write(x + '\n'))
summary_string = stream.getvalue()
stream.close()

# Define your other parameters
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
epochs = 10
training_accuracy = 0.95  # Example value
testing_accuracy = 0.90  # Example value

# Call the function to log everything
log_model_details(timestamp, summary_string, epochs, training_accuracy, testing_accuracy)

In [22]:
""""
# Define a Policy-Based RL Environment
class FraudDetectionEnv(gym.Env):
    def __init__(self):
        super(FraudDetectionEnv, self).__init__()
        self.action_space = gym.spaces.Discrete(n_actions)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(n_features + 1,))

    def step(self, action):
        # Implement the logic to handle actions and return the next state, reward, done, info
        pass

    def reset(self):
        # Reset the environment to an initial state
        pass

# Initialize the environment and the RL agent
env = FraudDetectionEnv()
model = PPO("MlpPolicy", env, verbose=1)

# Train the RL agent
model.learn(total_timesteps=10000)

# Example of using the trained autoencoder and RL agent in tandem
def make_decision(transaction_data):
    # Use autoencoder for anomaly detection
    reconstructed = autoencoder.predict(transaction_data)
    reconstruction_error = np.mean(np.abs(transaction_data - reconstructed), axis=1)

    # Include reconstruction error in the state for the RL agent
    state = np.concatenate([transaction_data, reconstruction_error[:, None]], axis=1)
    action = model.predict(state, deterministic=True)[0]
    
    return action

# Example transaction data (dummy data for illustration)
sample_transaction = np.random.rand(1, n_features)

# Make a decision on the transaction
decision = make_decision(sample_transaction)
print("Decision on transaction:", decision)
"""

'"\n# Define a Policy-Based RL Environment\nclass FraudDetectionEnv(gym.Env):\n    def __init__(self):\n        super(FraudDetectionEnv, self).__init__()\n        self.action_space = gym.spaces.Discrete(n_actions)\n        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(n_features + 1,))\n\n    def step(self, action):\n        # Implement the logic to handle actions and return the next state, reward, done, info\n        pass\n\n    def reset(self):\n        # Reset the environment to an initial state\n        pass\n\n# Initialize the environment and the RL agent\nenv = FraudDetectionEnv()\nmodel = PPO("MlpPolicy", env, verbose=1)\n\n# Train the RL agent\nmodel.learn(total_timesteps=10000)\n\n# Example of using the trained autoencoder and RL agent in tandem\ndef make_decision(transaction_data):\n    # Use autoencoder for anomaly detection\n    reconstructed = autoencoder.predict(transaction_data)\n    reconstruction_error = np.mean(np.abs(transaction_data - reconstructed),